## Import libs

In [20]:
import pandas as pd
import numpy as np
import os 

import seaborn as sns
import matplotlib.pyplot as plt

import pickle

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [21]:
import mlflow


mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("random-forest-hyperopt")

2024/05/31 14:17:25 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-hyperopt' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/2', creation_time=1717154245546, experiment_id='2', last_update_time=1717154245546, lifecycle_stage='active', name='random-forest-hyperopt', tags={}>

## Download data

In [ ]:
!python3 preprocess_data.py --raw_data_path "./data" --dest_path "./output"

## Q1. Install MLflow

### What's the version that you have?
2.13.0

In [3]:
!mlflow --version

mlflow, version 2.13.0


## Q2. Download and preprocess the data

### How many files were saved to OUTPUT_FOLDER?
4

In [5]:
!python preprocess_data.py --raw_data_path "./data" --dest_path "./output"

In [7]:
!ls "./output"

dv.pkl    test.pkl  train.pkl val.pkl


## Q3. Train a model with autolog

What is the value of the min_samples_split parameter:

    2


In [8]:
def load_pickle(filename: str):
    with open(filename, "rb") as f_in:
        return pickle.load(f_in)

In [18]:
def run_train(data_path: str):
    mlflow.sklearn.autolog()
    
    X_train, y_train = load_pickle(os.path.join(data_path, "train.pkl"))
    X_val, y_val = load_pickle(os.path.join(data_path, "val.pkl"))
    
    with mlflow.start_run():
        rf = RandomForestRegressor(max_depth=10, random_state=0)
        rf.fit(X_train, y_train)
        y_pred = rf.predict(X_val)

        with open('models/rand_forest.bin', 'wb') as f_out:
            pickle.dump(rf, f_out)
        
        mlflow.log_artifact(local_path="models/rand_forest.bin", artifact_path="models_pickle")

        rmse = mean_squared_error(y_val, y_pred, squared=False)


In [19]:
run_train('./output')

2024/05/31 14:10:37 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'


## Question 5. Tune model hyperparameters

In [22]:
!python3 hpo.py

100%|██████████| 15/15 [01:46<00:00,  7.08s/trial, best loss: 5.335419588556921]


best loss: 5.335419588556921

## Question 6.

In [36]:
!python3 register_model.py

2024/05/31 14:42:49 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-best-models' does not exist. Creating a new experiment.
2024/05/31 14:42:51 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2024/05/31 14:43:19 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2024/05/31 14:43:54 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2024/05/31 14:44:23 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2024/05/31 14:44:54 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
Successfully registered model 'ran

Best test_rmse is 5.567408012462019